# Toolkit for Reasoning with Hierarchies of Open-Textured Predicates

This notebook contains an implementation of the framework presented in "Reasoning with hierarchies of open-textured predicates", by Ilaria Canavotto and John Horty, proeedings of ICAIL, 2023 ([https://dl.acm.org/doi/10.1145/3594536.3595148](https://dl.acm.org/doi/10.1145/3594536.3595148)). 

The code is developed by Eric Pacuit, University of Maryland, epacuit@umd.edu 
 

In [2]:
from enum import Enum, auto
import random
from itertools import chain, combinations, product
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
from tqdm.notebook import tqdm
import seaborn as sns
import sys
from pyvis.network import Network
from IPython.display import display, HTML
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from itertools import product
from  tabulate import tabulate



## Helper Functions

In [3]:
def powerset(iterable):
    """
    Returns the powerset of the iterable: 
    powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)
    """
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

In [4]:
    
def display_concern(concern): 
    # display a concern, where a concern is a tuple of factors.
    print(str(concern[0]) + "/" + str(concern[1]))
    
def display_concerns(concerns, init_str=''):
    # display a set of concerns (possibly with an initial string)
    if type(concerns[0]) != tuple:
        print(init_str + "{" + ", ".join([str(f) for f in concerns]) + "}")
    else:
        print(init_str + "{" + ", ".join([str(c[0]) + '/' + str(c[1]) for c in concerns]) + "}")



## Factors

A **factor** is a predicate, or concept---often open-
textured---used to characterize a situation.  Some factors have contraries, in the traditional sense that two contrary factors cannot both apply in a particular situation.

In [5]:

class Factor(object): 

    def __init__(self, 
                 name, # name of the factor
                 has_contrary=True, # True when the factor has a contrary,
                 _contrary=None # the contrary factor
                 ): 
        
        if has_contrary: 

            if _contrary is None: # create the contrary factor
                self.name = f"f{name}+" if type(name) != tuple else f"f{name[0]}" 
                self._concern_idx = 0
                self.contrary =  Factor(name, 
                                        has_contrary=True,
                                        _contrary = self)
            elif _contrary is not None: # the contrary factor is provided
                self.name = f"f{name}-" if (type(name) != tuple and type(name) != list) else f"f{name[1]}" 
                self._concern_idx = 1
                self.contrary = _contrary
        else:  # the factor does not have a factor
            self.name = f"f{name}"
            self._concern_idx = None
            self.contrary = None
            
    @property
    def c(self): 
        # return the contrary of the factor (if there is no contrary, return None)
        return self.contrary
    
    def has_contrary(self): 
        # True if the factor has a contrary
        return self.contrary is not None
    
    def concern(self): 
        """
        Return the concern generated by the factor and its contrary.
        """
        return ((self, self.contrary) if self._concern_idx == 0 else (self.contrary, self)) if self.has_contrary() else None
    
    def display_concern(self):
        """
        Display the concern generated by the factor and its contrary.
        """
        
        if self.has_contrary() and self._concern_idx == 0: 
            print(str(self) + "/" + str(self.c))
        elif self.has_contrary() and self._concern_idx == 1: 
            print(str(self.c) + "/" + str(self))
        else: 
            print("")
    
    def __str__(self): 
        return str(self.name)
    
    def __eq__(self, other): 
        return self.name == other.name
    
    def __ne__(self, other): 
        return self.name != other.name
    
    def __hash__(self): 
        return hash(str(self.name))
    
def factory(with_contraries = None, without_contraries=None): 
    """
    Create factors from lists of names: with_contraries is the list of factor with contraries and without_contraries is the list of factors without contraries. 
    
    After running this function, for each name of a factor n, a variable fn is created and added to the global namespace. 
    """
    
    with_contraries = with_contraries if with_contraries is not None else list()
    without_contraries = without_contraries if without_contraries is not None else list()
    
    
    factors_with_contraries = dict()
    for n in with_contraries: 
        if type(n) == tuple or type(n) == list: 
            f = Factor(n)
            factors_with_contraries[f'f{n[0]}'] = f
            factors_with_contraries[f'f{n[1]}'] = f.c
        else: 
            factors_with_contraries[f'f{n}'] = Factor(n)
    factors_without_contraries = {f'f{n}': Factor(n, has_contrary=False) for n in without_contraries}
    module = sys.modules[__name__]
    for name, value in factors_with_contraries.items():
        setattr(module, name, value)
    for name, value in factors_without_contraries.items():
        setattr(module, name, value)

    print(f"Created {len(with_contraries)} {'factor variables ' if len(with_contraries) != 1 else 'factor variable'} with contraries and {len(without_contraries)} {'factor variables ' if len(with_contraries) != 1 else 'factor variable'} without contraries")



In [6]:
f1 = Factor(1, has_contrary=False)
print(f"The contrary of {f1} is ", f1.c)
f2 = Factor(2)
print(f"The contrary of {f2} is ", f2.c)
f2.display_concern()
fpi = Factor(('pi', 'delta'))
print(f"The contrary of {fpi} is ", fpi.c)
fpi.display_concern()


The contrary of f1 is  None
The contrary of f2+ is  f2-
f2+/f2-
The contrary of fpi is  fdelta
fpi/fdelta


In [7]:
factory(['a', 'b'], ['c'])

print(fa)
fa.display_concern()
print(fb)
fb.display_concern()
print(fc)
fc.display_concern()

Created 2 factor variables  with contraries and 1 factor variables  without contraries
fa+
fa+/fa-
fb+
fb+/fb-
fc



## Hierarchy

A **factor link** is a statement of the form $s \rightarrow t$ indicating that the presence of the factor $s$ in some situation directly favors a decision that $t$ holds as well, or simply that $s$ directly favors $t$. A **factor hierarchy** is a set $\mathcal{H}$ of factor links.

In [8]:

class Hierarchy(object): 
    
    def __init__(self, links): 
        """
        A hierarchy is defined by providing the list of factor links.
        """
        
        # the graph representing the hierarchy
        self.g = nx.DiGraph()
        self.g.add_edges_from(links)
        
        # the factors that are nodes in the graph
        self.nodes = self.g.nodes
        
        # all the factors that belong to the hierarchy: each factor in the graph and its contrary
        self.factors = list(self.nodes) + [f.c for f in self.nodes if f.has_contrary() and  f.c not in self.nodes]
        
        # the concern graph: a graph where each node is a concern and each edge is a link between two concerns.  This is useful for displaying the hierarchy
        link_to_node = lambda l: (l[0].concern(), l[1].concern()) if l[0].has_contrary() else (l[0], l[1].concern())
        self.concern_graph = nx.DiGraph()
        self.concern_graph.add_nodes_from([f.concern() if f.has_contrary() else f for f in self.g.nodes])
        self.concern_graph.add_edges_from([link_to_node(e) for e in links])
        
        if not self.is_acyclic(): 
            print("Error: The factor hierarchy is not acyclic.")
            
    def is_acyclic(self): 
        # return True if there is no cycle in the concern graph
        cycles = list(nx.simple_cycles(self.concern_graph))
        return len(cycles) == 0

    def is_abstract_factor(self, f):
        # An abstract factor is the target of some factor link in the hierarchy.
        return (f in self.g.nodes and len(self.g.in_edges(f)) != 0) or (f.c in self.g.nodes and len(self.g.in_edges(f.c)) != 0)

    def is_base_level_factor(self, f):
        # A base level factor has no contrary and is not the target of any factor link in the hierarchy.
        return not f.has_contrary() and f in self.g.nodes and len(self.g.in_edges(f)) == 0
    
    def is_intermediate_level_factor(self, f):
        # An intermediate level factor is an abstract factor such that the factor or its contrary is the source of some link in the hierarchy.
        return f.has_contrary() and self.is_abstract_factor(f) and (f in self.g.nodes and len(self.g.out_edges(f)) != 0) or (f.c in self.g.nodes and len(self.g.out_edges(f)) != 0)
    
    def is_top_level_factor(self, f):
        # The toplevel factor is an abstract factor that is not the source of any link in the hierarchy.
        return f.has_contrary() and self.is_abstract_factor(f) and (f not in self.g.nodes or len(self.g.out_edges(f)) == 0) and (f.c not in self.g.nodes or len(self.g.out_edges(f.c)) == 0)
    
    def is_concern(self, c):
        # Return True if c is a concern in the hierarchy.
        return self.is_abstract_factor(c[0]) and self.is_abstract_factor(c[1]) and c[0].c == c[1]
        
    def concerns(self):
        # Return all the concerns in the hierarchy. 
        return list(set([f.concern() for f in self.g.nodes if self.is_abstract_factor(f)]))

    def direct_favors_factor(self, f):
        # Return the last of factors that directly favor f, so all the factors that are the source of a link to f. 
        return list(self.g.predecessors(f)) if f in self.g.nodes else list()
    
    def all_directly_favor(self, factors, f): 
        # Return true if all the factors directly favor f.
        
        favor_list = self.direct_favors_factor(f)
        return all([_f in favor_list for _f in factors])
    
    def direct_favors(self, concern): 
        # Return the list of factors that directly favor either of the components of the concern.
        return self.direct_favors_factor(concern[0]) + self.direct_favors_factor(concern[1])
    
    def is_intermediate_level(self, concern): 
        # Return true if  concern is an intermediate level concern.
        return self.is_intermediate_level_factor(concern[0]) and self.is_intermediate_level_factor(concern[1])

    def is_top_level(self, concern): 
        # Return true if  concern is an top-level concern.
        return self.is_top_level_factor(concern[0]) and self.is_top_level_factor(concern[1])
    
    def intermediate_concerns(self): 
        # Return the list of intermediate level concerns.
        return list(set([f.concern() for f in self.g.nodes if self.is_intermediate_level_factor(f)]))
    
    def base_level_factors(self): 
        # Return the list of base level factors.
        return [f for f in self.g.nodes if self.is_base_level_factor(f)]
    
    def top_level_factors(self): 
        # Return the list of top level factors.
        return [f for f in self.factors if self.is_top_level_factor(f)]
    
    def issues(self): 
        # Return the list of issues in the hierarchy. An issue is a top-level concern.
        return list(set([f.concern() for f in self.g.nodes if self.is_top_level_factor(f)]))
          
    def is_standard(self): 
        # Returns True if the hierarchy is a *standard hierarchy*.  A hierarchy is standard if it is a single-issue hierarchy and all the factors in the hierarchy are either base level factors or top level factors.
        
        return self.is_single_issue() and all([f in self.base_level_factors() or f in self.top_level_factors() for f in self.factors])
        
    def favor(self, f): 
        # Return the list of factors that favor f and the list of factors that favor the contrary of f.

        favor_f = list()
        favor_f_c = list()

        if self.is_base_level_factor(f): 
            return list(), list()
        
        for t in self.direct_favors_factor(f): 
            favor_f.append(t)
            if t.has_contrary():
                favor_f_c.append(t.c)
            
        for t in self.direct_favors_factor(f.c): 
            
            if t.has_contrary():
                favor_f.append(t.c)
            favor_f_c.append(t)
        added_new_factors = True

        while added_new_factors: 
            
            new_favor_fs = list()
            new_favor_f_cs = list()
            added_new_factors = False
            for t in favor_f:  
                for u in self.direct_favors_factor(t): 
                    if u not in favor_f:
                        new_favor_fs.append(u)
                        added_new_factors = True
                    if u.has_contrary() and u.c not in favor_f_c:
                        new_favor_f_cs.append(u.c)
                        added_new_factors = True
                for u in self.direct_favors_factor(t.c): 
                    if u not in favor_f_c:
                        new_favor_f_cs.append(u)
                        added_new_factors = True
                    if u.has_contrary() and u.c not in favor_f: 
                        new_favor_fs.append(u.c)
                        added_new_factors = True
            
            favor_f = list(set(favor_f + new_favor_fs))
            favor_f_c = list(set(favor_f_c + new_favor_f_cs))
        return favor_f, favor_f_c
           
    def is_uniform(self): 
        # Return true if the factor hierarchy is uniform (see Definition 3).
         
        for f in self.nodes(): 
            favor_f, favor_f_c = self.favor(f)
            if set(favor_f).intersection(set(favor_f_c)) != set(): 
                return False
        return True
    
    def is_single_issue(self): 
        # Return True if this is a single-issue hierarchy (there is only one concern that is the root of the hierarchy).
         
        return len(self.issues()) == 1
    
    def degree(self, factor_or_concern): 
        # Return the degree of the factor or concern.
        
        
        if type(factor_or_concern) == tuple: # input is a concern
            f = factor_or_concern[0]
        else: # input is a factor
            f = factor_or_concern
        if self.is_base_level_factor(f): 
            return 0
        return max([self.degree(_f) for _f in self.direct_favors(f.concern())]) + 1
    
    def length(self): 
        # The length of a hierarchy is the maximum degree of the issues
        
        return max([self.degree(i) for i in self.issues()])

    def concerns_at_degree(self, d): 
        # Return a list of all concerns at degree d.
 
        return list(set([f.concern() for f in self.nodes if self.degree(f) == d])) if d > 0 else list(set([f for f in self.nodes if self.degree(f) == d]))
    
    def reason_targets(self, fs): 
        # Return the list of abstract factors that fs supports.
        
        pot_targets = list()
        for f in self.nodes: 
            if self.is_abstract_factor(f) and self.all_directly_favor(fs, f): 
                pot_targets.append(f)
                
        return pot_targets
    
    def get_potential_reasons(
        self, 
        factors, 
        target=None, 
        allow_empty_reason=False):  
        """Return all potential reasons for  the target."""
        pot_reasons = list()
        
        if target is None: 
            for fs in powerset(factors): 
                if len(fs) > 0: 
                    pot_reasons += [Reason(fs, self, t) for t in self.get_reason_targets(fs)]
                if allow_empty_reason and len(fs) == 0: 
                    pot_reasons += [Reason(fs, self, t) for t in self.get_reason_targets(fs)]
        else: 
            for fs in powerset(factors): 
                if len(fs) > 0 and self.all_directly_favor(fs, target): 
                    pot_reasons.append(Reason(fs, self, target))
                if allow_empty_reason and len(fs) == 0: 
                    pot_reasons.append(Reason(fs, self, target))
        return pot_reasons

    def raised_concerns(self, fact_sit, degree=None): 
        """
        Return the list of concerns that are raised by the fact situation fact_sit. If degree is not None, only return the concerns at the given degree.
        """
        concerns = list() 
        for f in fact_sit: 
            if f in self.g.nodes: 
                for _f in self.g.successors(f): 
                    
                    if degree is None or self.degree(_f) == degree:
                        concerns.append(_f.concern())
        return list(set(concerns))
                

def display_with_selected_factor(hierarchy = None, factor = None): 
    """
    Display the hierarchy with the selected factor in blue, and all factors that favor the selected factor in green.   Red arrows indicate that the factor is the contrary of another factor.

    **Note**: This function requires the pyvis library and only works when the notebook is run in a browser.
    """
    nx_g = nx.DiGraph()
        
    if factor is not  None:
        favor, favor_c = hierarchy.favor(factor)
    else: 
        favor = list()
            
    font_string = "14px arial white"
    for f in hierarchy.g.nodes:
        if f.has_contrary():
            if factor is not None and f == factor:
                nx_g.add_node(str(f), color="blue", font=font_string,   shape="box")
                nx_g.add_node(str(f.c), color="gray", font=font_string,   shape="box")
            elif factor is not None and f.c == factor:
                nx_g.add_node(str(f.c), color="blue", font=font_string,   shape="box")
                nx_g.add_node(str(f), color="gray", font=font_string,   shape="box")
                    
            else: 
                nx_g.add_node(str(f), color="gray", font= font_string,   shape="box")
                nx_g.add_node(str(f.c), color="gray", font=font_string,   shape="box")
                    
            nx_g.nodes[str(f)]['level'] = hierarchy.degree(f)
            nx_g.nodes[str(f)]['title'] = str(f)
            nx_g.nodes[str(f.c)]['level'] = hierarchy.degree(f)
            nx_g.nodes[str(f.c)]['title'] = str(f)
        else: 
            if f in favor: 
                nx_g.add_node(str(f), color="green", font=font_string,   shape="box")
            else: 
                nx_g.add_node(str(f), color="gray", font=font_string,   shape="box")
                    
            nx_g.nodes[str(f)]['level'] = hierarchy.degree(f)
            nx_g.nodes[str(f)]['title'] = str(f)

    for f in favor: 
        nx_g.add_node(str(f), color="green", font=font_string, shape="box")
        nx_g.nodes[str(f)]['title'] = f"{str(f)} favors {str(factor)}"
            
    for f in hierarchy.g.nodes: 
        if f.has_contrary(): 
            nx_g.add_edge(str(f), str(f.c), color="red", title=f"{f} and {f.c} are contraries")
            nx_g.add_edge(str(f.c), str(f), color="red")


    for e in hierarchy.g.edges: 
        nx_g.add_edge(str(e[0]), str(e[1]), title=f"{e[0]} directly favors {e[1]}")
            
    nt = Network('600px', '100%', notebook=True, layout=True,  directed =True)

    nt.set_options('''var options = {
           "configure": {
                "enabled": false
           },
           "node" : {
               "font": {
                   "color": "white"
               }
           },
          "interaction": {
            "zoomView": false
          },
          "layout": {
            "hierarchical": {
              "enabled": true,
              "levelSeparation": -150,
              "direction": "DU"
            }
          }
        }
        ''')
    nt.from_nx(nx_g) 
    display(nt.show('nx_g.html'))
    


In [9]:
factory(
    [101, 102, 105, 122, 106, 108], # factors with contraries 
    [1, 6, 15, 16] #factors without contraries
)
      
links = [
    (f6, f102), 
    (f1, f122.c), 
    (f122, f102), 
    (f102, f101), 
    (f105, f101.c),
    (f106, f105),
    (f108, f105),
    (f15, f106.c),
    (f16, f108), 
]

h = Hierarchy(links)


Created 6 factor variables  with contraries and 4 factor variables  without contraries


In [10]:
factors_for_display = list()
for c in h.issues() + h.intermediate_concerns(): 
    factors_for_display.append((str(c[0]), c[0]))
    factors_for_display.append((str(c[1]), c[1]))
interact(display_with_selected_factor, 
         hierarchy=fixed(h), 
         factor=factors_for_display);


interactive(children=(Dropdown(description='factor', options=(('f101+', <__main__.Factor object at 0x2b1a45250…

## Fact Situation, Reason, Rule, and Decision

In [11]:
class FactSituation(object): 
    """
    A fact situation is a set of factors from a hierarchy. 
    """

    def __init__(self, hierarchy, factors):
        assert all([f in hierarchy.factors for f in factors]), "All factors must be in the hierarchy."

        self.hierarchy = hierarchy
        self.factors = set(factors) 

    def is_base_level(self): 
        return all([f in self.hierarchy.base_level_factors() for f in self.factors])

    def __contains__(self, f):
        assert isinstance(f, Factor), f"{f} must be a factor."
        return f in self.factors
    
    def __iter__(self):
        return iter(self.factors)

    def __eq__(self, other):
        return self.factors == other.factors and all([f in self.factors for f in other.factors]) and all([f in other.factors for f in self.factors])
    
    def __add__(self, other): 
        if isinstance(other, FactSituation): 
            return FactSituation(self.hierarchy, self.factors.union(other.factors))
        elif isinstance(other, list):
            return FactSituation(self.hierarchy, self.factors.union(set(other)))
    def __str__(self): 
        return "{" + ",".join(sorted([str(f) for f in self.factors])) + "}"
    
    def __hash__(self): 
        return hash(tuple([str(f) for f in self.factors])) if len(self.factors) > 0 else hash(tuple([]))

class Reason(object): 
    '''
    A reason in a hierarchy is a set of factors that directly favor an abstract factor.
    '''
    def __init__(self, factors, hierarchy, target): 
        
        assert hierarchy.is_abstract_factor(target), f"The target of a reason must be an abstract factor: {str(target)} is not an abstract reason."
        assert hierarchy.all_directly_favor(factors, target), "The set of factors must directly favor the target. "
        
        self.hierarchy = hierarchy
        self.factors = factors
        self.target = target
        
    @property
    def reason_for(self): 
        return self.target
    
    def is_empty(self): 
        return len(self.factors) == 0

    def holds_in(self, fact_sit):
        """
        Return True if the reason holds in the fact situation fact_sit
        """
        return all([f in fact_sit for f in self.factors])  
              
    def __eq__(self, other):
        return self.target == other.target and all([f in self.factors for f in other.factors]) and all([f in other.factors for f in self.factors])

    def __le__(self, other):
        return self.target == other.target and all([f in other.factors for f in self.factors])


    def __lt__(self, other):
        return self.__le__(other) and not self.__eq__(other)

    def __ge__(self, other):
        
        return self.target == other.target and all([f in self.factors for f in other.factors])

    def __gt__(self, other):
        return self.__ge__(other) and not self.__eq__(other)

    def __str__(self): 
        return "{" + ",".join([str(f) for f in self.factors]) + "}" + f":{self.target}" if len(self.factors) > 0 else  "{" + "}" + f":{self.target}"
    
    def __hash__(self): 
        return hash(tuple([str(f) for f in self.factors])) if len(self.factors) > 0 else hash(tuple([f":{str(self.target)}"]))

def reasons_from_factors(factors, hierarchy, target, include_empty_reason=False): 
    """
    Given a list of factors all directly favoring target, return all the reasons from this list of factors. 
    """
    reasons = list()
    for fs in powerset(factors): 
        if len(fs) > 0: 
            reasons.append(Reason(fs, hierarchy, target))
        if include_empty_reason and len(fs) == 0: 
            reasons.append(Reason(fs, hierarchy, target))
    return reasons

class Rule(object): 
    """
    A rule is a reason with a target.  A rule is applicable to a fact situation if the premise of the rule holds in the fact situation.
    """
    def __init__(self, reason): 
        self.premise = reason
        self.conclusion = reason.target
        self.hierarchy = reason.hierarchy
            
    def is_applicable(self, fact_sit):
        """
        Return True if the premise of the rule holds in the fact situation fact_sit
        """
        return self.premise.holds_in(fact_sit)
    
    def __str__(self): 
        factors = "{" + ",".join([str(f) for f in self.premise.factors]) + "}" 
        return f"{factors}-->{self.conclusion}"
        
        
class Decision(object): 
    """
    A decision is a fact situation and a rule.  A decision is applicable to a fact situation if the premise of the rule holds in the fact situation."""
    def __init__(self, fact_sit, rule):

        assert rule.is_applicable(fact_sit), f"The rule {str(rule)} is not applicable to the fact situation {str(fact_sit)}"

        self.fact_sit = fact_sit
        self.hierarchy = fact_sit.hierarchy
        self.rule = rule
        self.outcome = rule.conclusion
                
    @property
    def facts(self): 
        return self.fact_sit
    
    def prefers(self, v, u): #induced_priority
        """
        Return True if the decision prefers u to v:

        1. u and v are must be reasons for opposites sides of a concern in the hierarchy, 
        2. v must be a reason for the outcome of the decision, and 
        3. u must hold in the fact situation, and the premise of the rule must be a subset of v.
        """
        
        return self.hierarchy.is_concern((u.reason_for, v.reason_for)) and self.outcome == u.reason_for and v.holds_in(self.fact_sit) and self.rule.premise <= u

    def conflicts(self, other_decision): 
        # Return True if the decision conflicts with the other decision. 
        
        return self.outcome.c == other_decision.outcome
    
    def __str__(self): 
        factors = "{" + ",".join([str(f) for f in self.fact_sit]) + "}" 
        return "<" + factors + ", " + str(self.rule) + ", " + str(self.outcome) + ">"
    

In [12]:

factory(
    ['p', 'q', 'r', ('pi', 'delta')], 
    [1, 2, 3, 4, 5, 6])

factors = [f1, f2, f3, f4, f5, f6, fp, fq, fr, fpi]

links = [
    (f1, fp), 
    (f2, fp), 
    (f3, fp.c), 
    (f3, fq.c), 
    (f4, fr), 
    (f5, fr.c),
    (f6, fr.c), 
    (fp, fq), 
    (fq, fpi), 
    (fr.c, fpi.c)
    ]

h = Hierarchy(links)

print(h.is_concern((f1, f2)))
print(h.is_concern((fp, fp.c)))
print(h.is_concern((fp.c, fp)))
print(h.is_concern((fp.c, fq)))

fs1 = FactSituation(h, [f1, f2, f3])
print(fs1)
r1 = Reason([f1, f2], h, fp)

print(r1)

print(r1.holds_in(fs1))

for c in h.concerns(): 
    display_concern(c)
    for r in reasons_from_factors(h.direct_favors_factor(c[0]), h, c[0]): 
        print(r)
    for r in reasons_from_factors(h.direct_favors_factor(c[1]), h, c[1]): 
        print(r)


Created 4 factor variables  with contraries and 6 factor variables  without contraries
False
True
True
False
{f1,f2,f3}
{f1,f2}:fp+
True
fp+/fp-
{f1}:fp+
{f2}:fp+
{f1,f2}:fp+
{f3}:fp-
fpi/fdelta
{fq+}:fpi
{fr-}:fdelta
fr+/fr-
{f4}:fr+
{f5}:fr-
{f6}:fr-
{f5,f6}:fr-
fq+/fq-
{fp+}:fq+
{f3}:fq-


In [13]:
factory(['p', 'q', 'r', ('pi', 'delta')], [1, 2, 3, 4, 5, 6])

factors = [f1, f2, f3, f4, f5, f6, fp, fq, fr, fpi]

links = [(f1, fp), (f2, fp), (f3, fp.c), (f3, fq.c), (f4, fr), (f5, fr.c),(f6, fr.c), (fp, fq), (fq, fpi), (fr.c, fpi.c)]

h = Hierarchy(links)

factors_for_display = list()
for c in h.issues() + h.intermediate_concerns(): 
    factors_for_display.append((str(c[0]), c[0]))
    factors_for_display.append((str(c[1]), c[1]))
interact(display_with_selected_factor, 
         hierarchy=fixed(h), 
         factor=factors_for_display);



Created 4 factor variables  with contraries and 6 factor variables  without contraries


interactive(children=(Dropdown(description='factor', options=(('fpi', <__main__.Factor object at 0x2b1ad9dd0>)…

In [14]:
for d in [0, 1, 2, 3]: 
    print(f"Concerns at degree {d}")
    display_concerns(h.concerns_at_degree(d))
    print()

print()

fact_sit1 = FactSituation(h, [f1, f3, f5])

print(f"The concerns raised by {fact_sit1} are: ")

display_concerns(h.raised_concerns(fact_sit1))

print(f"The concerns of degree 1 raised by {fact_sit1} are: ")
display_concerns(h.raised_concerns(fact_sit1, degree=1))
print()
d1 = Decision(fact_sit1, Rule(Reason([f1], h, fp)))
print("Rule d1: ", d1)

d2 = Decision(fact_sit1, Rule(Reason([f5], h, fr.c)))
print("Rule d2: ", d2)

Concerns at degree 0
{f5, f4, f1, f6, f3, f2}

Concerns at degree 1
{fp+/fp-, fr+/fr-}

Concerns at degree 2
{fq+/fq-}

Concerns at degree 3
{fpi/fdelta}


The concerns raised by {f1,f3,f5} are: 
{fp+/fp-, fr+/fr-, fq+/fq-}
The concerns of degree 1 raised by {f1,f3,f5} are: 
{fp+/fp-, fr+/fr-}

Rule d1:  <{f3,f5,f1}, {f1}-->fp+, fp+>
Rule d2:  <{f3,f5,f1}, {f5}-->fr-, fr->


## Opinion

In [15]:
class Opinion(object): 
    """
    A *resolution* of a concern based on a fact situation is a decision for one side or the other of the concern, together with a rule justifying the decision that is applicable in that fact situation. An opinion for a hierarchy is a set of decisions which is a resolution for each concern in the hierarchy based on a fact situation.
    """
    def __init__(self, hierarchy, fact_sit): 
        
        all([f in hierarchy.base_level_factors() for f in fact_sit]), "The fact situation must be base level factors in the hierarchy."
        
        self.hierarchy = hierarchy
        self.fact_sit = fact_sit
        
        self.resolutions = {d: list() for d in range(1, self.hierarchy.length() + 1)}
        
    def set_decisions(self, decisions, degree): 
        assert len(decisions) > 0, "Cannot add an empty list of decisions."
        self.resolutions[degree] = decisions

    def get_decision_for(self, concern): 
        # Returns the decision for the concern.

        for d in self.resolutions[self.hierarchy.degree(concern)]:
            if d.outcome == concern[0] or d.outcome == concern[1]: 
                return d
        return None

    def supports(self, f): 
        # Returns True if the opinion is complete, f is the issue of the hierarchy, and the opinion supports f. 
        
        return  self.is_complete() and self.resolutions[self.hierarchy.length()][0].outcome == f

    def supported_factor(self): 
        # Return the issue that is supported by the opinion.

        if self.is_complete(): 
            return self.resolutions[self.hierarchy.length()][0].outcome
        else:
            return None
            
    def merge(self): 
        # Return all the decision in the opinion. 

        return [d for res in self.resolutions.values() for d in res]

    def factors(self, degree=None): 
        # Return all the factors triggered by the decisions up to degree (if None, return all factors). 
        
        fs = [f for f in self.fact_sit]
        for deg in range(1, self.hierarchy.length()+1): 
            if degree is None or deg <= degree: 
                fs += [d.outcome for d in self.resolutions[deg]]
                
        return FactSituation(self.hierarchy, fs)
    
    def is_complete(self, degree = None): 
        # Return True if the opinion has a complete set of resolutions (for degree if set).

        if degree is not None: 
            
            outcomes = [d.outcome for d in self.resolutions[degree]]
            if any([c[0] not in outcomes and c[1] not in outcomes for c in self.hierarchy.concerns_at_degree(degree)]): 
                return False
        else: 
            for deg, decisions in self.resolutions.items(): 
                outcomes = [d.outcome for d in decisions]
                if any([c[0] not in outcomes and c[1] not in outcomes for c in self.hierarchy.concerns_at_degree(deg)]): 
                    return False
        return True
    
    def make_decisions(self, degree): 
        # Make decisions for all concerns raised at degree.  This is non-deterministic. 
         
        curr_factors = self.factors(degree=degree - 1)
        
        raised_concerns = self.hierarchy.raised_concerns(curr_factors, degree=degree) 
            
        decisions = list()
        for c in raised_concerns: 

            # all the reasons for c in the hierarchy
            reasons_for = self.hierarchy.get_potential_reasons(curr_factors, target=c[0])

            # all the reasons against c in the hierarchy
            reasons_against = self.hierarchy.get_potential_reasons(curr_factors, target=c[1])

            # randomly choose a decision for the concern c
            decisions.append(Decision(curr_factors, Rule(random.choice(reasons_for + reasons_against))))
            
        self.resolutions[degree] = decisions
        return decisions
       
    def generate(self): 
        """
        Generate an opinion for the hierarchy given the fact situation. This is non-deterministic. 
        """
        
        for d in range(1, self.hierarchy.length() + 1): 
            self.make_decisions(d)
    
    def display(self): 
        # Display the opinion by listing all the decisions for each concern at each degree.
        for degree, ds in self.resolutions.items(): 
            print(f"The decisions for degree {degree}: ")
            
            if len(ds) == 0: 
                print("\tNone")
            else: 
                for d in ds: 
                    print("\t", d)

                    
    def show(self): 
        # Display the opinion by highlighting all the decisions in the hierarchy.
        nx_g = nx.DiGraph()
        
        font_string = "14px arial white"
        for f in self.hierarchy.g.nodes:
            if f.has_contrary():
                nx_g.add_node(str(f), color="gray", font= font_string,   shape="box")
                nx_g.add_node(str(f.c), color="gray", font=font_string,   shape="box")
                    
                nx_g.nodes[str(f)]['level'] = self.hierarchy.degree(f)
                nx_g.nodes[str(f)]['title'] = str(f)
                nx_g.nodes[str(f.c)]['level'] = self.hierarchy.degree(f)
                nx_g.nodes[str(f.c)]['title'] = str(f)
            else: 
                nx_g.add_node(str(f), color="gray", font=font_string,   shape="box")
                    
                nx_g.nodes[str(f)]['level'] = self.hierarchy.degree(f)
                nx_g.nodes[str(f)]['title'] = str(f)

        for deg, res in self.resolutions.items(): 
            for dec in res: 
                for f in dec.rule.premise.factors: 
                    nx_g.add_node(str(f), color="blue", font=font_string, shape="box")
            
        for f in self.hierarchy.g.nodes: 
            if f.has_contrary(): 
                nx_g.add_edge(str(f), str(f.c), color="red", title=f"{f} and {f.c} are contraries")
                nx_g.add_edge(str(f.c), str(f), color="red")


        for e in self.hierarchy.g.edges: 
            nx_g.add_edge(str(e[0]), str(e[1]), title=f"{e[0]} directly favors {e[1]}")
            
        nt = Network('600px', '100%', notebook=True, layout=True,  directed =True)

        nt.set_options('''var options = {
           "configure": {
                "enabled": false
           },
           "node" : {
               "font": {
                   "color": "white"
               }
           },
          "interaction": {
            "zoomView": false
          },
          "layout": {
            "hierarchical": {
              "enabled": true,
              "levelSeparation": -150,
              "direction": "DU"
            }
          }
        }
        ''')
        nt.from_nx(nx_g) 
        display(nt.show('nx_g.html'))
                    


In [16]:
fact_sit1 = FactSituation( h, [f1, f3, f4, f5])
o = Opinion(h, fact_sit1)

print(f"Concerns of degree 1 raised by {fact_sit1}")
display_concerns(h.raised_concerns(fact_sit1, degree=1))
print()
d1 = Decision(fact_sit1, Rule(Reason([f1], h, fp)))
print("decision 1: ", d1)

d2 = Decision(fact_sit1, Rule(Reason([f5], h, fr.c)))
print("decision 2: ", d2)
o.set_decisions([d1, d2], 1)

print()
o.display()
print()
print("Is the opinion complete? ", o.is_complete())
print("Is the opinion for degree 1 complete? ", o.is_complete(degree=1))

# print()
Y_0 = fact_sit1 
Y_1 = Y_0 + [d1.outcome, d2.outcome]
print(f"Concerns of degree 2 raised by {Y_1}")
display_concerns(h.raised_concerns(Y_1, degree=2))
print()
print(o.factors(degree=1))
d3 = Decision(o.factors(degree=1), Rule(Reason([fp], h, fq)))
print("decision 3: ", d3)

o.set_decisions([d3], 2)

print()
o.display()
print()
print("Is the opinion complete? ", o.is_complete())
print("Is the opinion for degree 1 complete? ", o.is_complete(degree=1))
print("Is the opinion for degree 2 complete? ", o.is_complete(degree=2))

Y_2 = Y_1 + [d3.outcome]

print(f"Concerns of degree 2 raised by {Y_2}")
display_concerns(h.raised_concerns(Y_2, degree=2))

print()
d4 = Decision(o.factors(degree=2), Rule(Reason([fq], h, fpi)))
print("decision 4: ", d4)

o.set_decisions([d4], 3)

print()
o.display()
print()
print("Is the opinion complete? ", o.is_complete())
print("Is the opinion for degree 1 complete? ", o.is_complete(degree=1))
print("Is the opinion for degree 2 complete? ", o.is_complete(degree=2))
print("Is the opinion for degree 3 complete? ", o.is_complete(degree=3))

print(o.supported_factor())
print()


Concerns of degree 1 raised by {f1,f3,f4,f5}
{fp+/fp-, fr+/fr-}

decision 1:  <{f3,f5,f1,f4}, {f1}-->fp+, fp+>
decision 2:  <{f3,f5,f1,f4}, {f5}-->fr-, fr->

The decisions for degree 1: 
	 <{f3,f5,f1,f4}, {f1}-->fp+, fp+>
	 <{f3,f5,f1,f4}, {f5}-->fr-, fr->
The decisions for degree 2: 
	None
The decisions for degree 3: 
	None

Is the opinion complete?  False
Is the opinion for degree 1 complete?  True
Concerns of degree 2 raised by {f1,f3,f4,f5,fp+,fr-}
{fq+/fq-}

{f1,f3,f4,f5,fp+,fr-}
decision 3:  <{f5,fr-,f4,f1,fp+,f3}, {fp+}-->fq+, fq+>

The decisions for degree 1: 
	 <{f3,f5,f1,f4}, {f1}-->fp+, fp+>
	 <{f3,f5,f1,f4}, {f5}-->fr-, fr->
The decisions for degree 2: 
	 <{f5,fr-,f4,f1,fp+,f3}, {fp+}-->fq+, fq+>
The decisions for degree 3: 
	None

Is the opinion complete?  False
Is the opinion for degree 1 complete?  True
Is the opinion for degree 2 complete?  True
Concerns of degree 2 raised by {f1,f3,f4,f5,fp+,fq+,fr-}
{fq+/fq-}

decision 4:  <{f5,fr-,f4,f1,fp+,f3,fq+}, {fq+}-->fpi, fpi>

In [17]:
fact_sit1 = FactSituation(h, [f1, f2, f3, f4, f5])

o = Opinion(h, fact_sit1)
o.generate()
o.display()
print("The opinion is complete: ", o.is_complete())
print("The factor supported by the decision: ", o.supported_factor())
print("The decisions in the opinion:")
for d in o.merge(): 
    print(d)

The decisions for degree 1: 
	 <{f5,f4,f1,f3,f2}, {f3}-->fp-, fp->
	 <{f5,f4,f1,f3,f2}, {f5}-->fr-, fr->
The decisions for degree 2: 
	 <{f5,fr-,f4,fp-,f1,f3,f2}, {f3}-->fq-, fq->
The decisions for degree 3: 
	 <{f5,fr-,f4,fp-,fq-,f1,f3,f2}, {fr-}-->fdelta, fdelta>
The opinion is complete:  True
The factor supported by the decision:  fdelta
The decisions in the opinion:
<{f5,f4,f1,f3,f2}, {f3}-->fp-, fp->
<{f5,f4,f1,f3,f2}, {f5}-->fr-, fr->
<{f5,fr-,f4,fp-,f1,f3,f2}, {f3}-->fq-, fq->
<{f5,fr-,f4,fp-,fq-,f1,f3,f2}, {fr-}-->fdelta, fdelta>


## Case and Case Base

In [18]:
class Case(object): 
    """
    A case is a fact situation and an opinion. 
    """
    def __init__(self,  fact_situation, opinion): 

        assert opinion.is_complete(), "The opinion must be complete."
        assert fact_situation.is_base_level(), "The fact situation must be base level set of factors."
        
        self.hierarchy = fact_situation.hierarchy
        self.opinion = opinion
        self.facts = fact_situation
        self.outcome = opinion.supported_factor()

    def get_decision_for(self, concern):
        """
        Return the decision for the concern.
        """
        return self.opinion.get_decision_for(concern)

    def prefers(self, v, u): 
        """
        Return True is u has a higher priority than v.
        """

        return any([d.prefers(v, u) for d in self.opinion.merge()])
        
    def display(self): 
        print(f"Facts: {self.facts}")
        print(f"Opinion: ")
        self.opinion.display()
        print(f"Outcome: {self.outcome}")

class CaseBase(object): 
    """
    A case base is a set of cases.
    """
    def __init__(self, hierarchy, cases = None): 

        self.cases = cases if cases is not None else []
        self.hierarchy = hierarchy

    def add(self, c): 
        self.cases.append(c)

    def prefers(self, v, u): 
        """
        Return True is u has a higher priority than v according to the current cases.
        """
        return any([c.prefers(v, u) for c in self.cases])

    def is_consistent(self, case): 
        """
        Return True if the case is consistent with the current cases.  For each concern, the decision from the case must be consistent with the decisions from the current cases.
        """

        for d in range(1, self.hierarchy.length()+1): 
            for concern in self.hierarchy.concerns_at_degree(d): 
                dec_from_case = case.get_decision_for(concern)
                all_cb_decisions = [c.get_decision_for(concern) for c in self.cases]

                cb_decisions_pot_conflict = [d for d in all_cb_decisions if d.outcome == dec_from_case.outcome.c]

                if any([self.prefers(dec_from_case.rule.premise, d.rule.premise) and self.prefers(d.rule.premise, dec_from_case.rule.premise) for d in cb_decisions_pot_conflict]) : 

                    #print("NOT CONSISTENT")
                    #print(f"Decision from case: {dec_from_case}")
                    #for d in cb_decisions_pot_conflict: 
                    #    if self.prefers(dec_from_case.rule.premise, d.rule.premise) and self.prefers(d.rule.premise, dec_from_case.rule.premise): 
                    #        print(f"CONFLICTING Decision from case base: {d}")    

                    return False
        return True
        
    def is_consistent_brute_force(self, case = None): 
        """
        Return True if the case is consistent with the current cases.

        If case is None, then return True if the current case base is consistent.

        Note: If the case base is inconsistent, then this will return False.
        """

        other_prefers = self.prefers if case is None else case.prefers

        for c in self.hierarchy.concerns():

            reasons_for_c0 = reasons_from_factors(self.hierarchy.direct_favors_factor(c[0]), self.hierarchy, c[0])

            reasons_for_c1 = reasons_from_factors(self.hierarchy.direct_favors_factor(c[1]), self.hierarchy, c[1])

            for r_for_c0 in reasons_for_c0: 
                for r_for_c1 in reasons_for_c1: 
                    if (self.prefers(r_for_c0, r_for_c1) and other_prefers(r_for_c1, r_for_c0)) or (self.prefers(r_for_c1, r_for_c0) and other_prefers(r_for_c0, r_for_c1)): 
                        return False
        return True

    def display(self): 
        """
        Display all the CaseBase as a table where each column is labeled by a case and the rows are labeled by the concerns.  Each entry is the outcome of the decision from the case for the concern.
        """

        decisions = []

        # get all concerns in the hierarchy
        all_concerns = [f"{c[0]}/{c[1]}" for d in range(1, self.hierarchy.length()+1) for c in self.hierarchy.concerns_at_degree(d)]

        # reverse the concerns so degree 1 are at the bottom of the table
        all_concerns.reverse()

        # get all the decisions for each case
        case_decisions = {" ": all_concerns}
        for cnum, case in enumerate(self.cases): 
            decisions = []
            for d in range(1, case.hierarchy.length()+1): 
                for c in case.hierarchy.concerns_at_degree(d): 
                    decisions.append(case.get_decision_for(c).outcome)

            decisions.reverse()
            case_decisions[f"c{cnum}"] = list(map(str,decisions))
        print(tabulate(
            case_decisions, 
            headers="keys",
            tablefmt="rounded_outline")
            )


In [24]:

fact_sits = [
    FactSituation( h, [f1, f4, f5]),
    FactSituation( h, [f1, f2, f3]),
    FactSituation( h, [f1, f2, f3, f4]), 
]
case_base = CaseBase(h)
for fs in fact_sits:
    print(f"Finding opinion for {fs}")
    for i in range(10): 
        o = Opinion(h, fact_sit1)
        o.generate()
        if o.is_complete():
            #o.display()
            case = Case(fs, o)
            if case_base.is_consistent(case):
                #print(case_base.is_consistent_brute_force(case))
                case_base.add(case)
                break
case_base.display()


Finding opinion for {f1,f4,f5}
Finding opinion for {f1,f2,f3}
Finding opinion for {f1,f2,f3,f4}
╭────────────┬──────┬────────┬────────╮
│            │ c0   │ c1     │ c2     │
├────────────┼──────┼────────┼────────┤
│ fpi/fdelta │ fpi  │ fdelta │ fdelta │
│ fq+/fq-    │ fq+  │ fq-    │ fq+    │
│ fr+/fr-    │ fr-  │ fr-    │ fr-    │
│ fp+/fp-    │ fp+  │ fp-    │ fp+    │
╰────────────┴──────┴────────┴────────╯
